# Boston housing

## Resources

- https://towardsdatascience.com/using-python-to-find-myself-a-rental-home-a0b1bf6f02e
- https://github.com/johnludwigm/PyApartments
- https://github.com/adinutzyc21/apartments-scraper
- https://www.dataquest.io/blog/apartment-finding-slackbot/

In [ ]:
# import requests as rq
# from bs4 import BeautifulSoup
# from urllib.request import urlopen,urlretrieve
# urlopen(url)
# urlretrieve(url, "page1.html")
# page = rq.get(url)
# soup = BeautifulSoup(page.content, "html.parser")
# soup.find(class_="rentInfoDetail")

## Selenium

### Installation

| Browser | Webdriver |
|---------|-----------------------------------------------|
| Chrome: |	https://sites.google.com/chromium.org/driver/ |
| Edge: |	https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/ |
| Firefox: |	https://github.com/mozilla/geckodriver/releases |

### Simple usage
https://selenium-python.readthedocs.io/getting-started.html#simple-usage

### Locating elements
https://selenium-python.readthedocs.io/locating-elements.html

Example usage:
```python
from selenium.webdriver.common.by import By

driver.find_element(By.XPATH, '//button[text()="Some text"]')
driver.find_elements(By.XPATH, '//button')
```

These are the attributes available for By class:
```python
ID = "id"
XPATH = "xpath"
LINK_TEXT = "link text"
PARTIAL_LINK_TEXT = "partial link text"
NAME = "name"
TAG_NAME = "tag name"
CLASS_NAME = "class name"
CSS_SELECTOR = "css selector"
```

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
driver = webdriver.Chrome("chromedriver.exe")

C:\Users\badmo\AppData\Local\Temp\ipykernel_11036\3684089475.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe")


## Manual extract

In [84]:
url = "https://www.apartments.com/5-donnybrook-rd-boston-ma/rhyk6z3/"

In [85]:
driver.get(url)

In [86]:
# price = driver.find_element_by_class_name("rentInfoDetail")
price = driver.find_element(by=By.CLASS_NAME, value="rentInfoDetail")
price.text

'$2,750'

In [87]:
t = ["price","bedrooms","bathrooms","surface"]

In [88]:
info = driver.find_elements(by=By.CLASS_NAME, value="rentInfoDetail")
infos = [i.text for i in info]
dict(zip(t,infos))

{'price': '$2,750',
 'bedrooms': '3 bd',
 'bathrooms': '2 ba',
 'surface': '999 sq ft'}

In [89]:
address = driver.find_element(by=By.CLASS_NAME, value="propertyAddressContainer")
street = address.find_element(by=By.TAG_NAME,value="span").text
street

'5 Donnybrook Rd'

In [94]:
address.text.split("\n")[0]

'5 Donnybrook Rd, Boston, MA 02135'

In [90]:
[i.text for i in address.find_elements(by=By.TAG_NAME,value="span")]

['5 Donnybrook Rd', 'Boston', 'MA 02135', 'MA', '02135', 'Oak Square', '']

In [91]:
import pandas as pd
df = pd.DataFrame([infos],columns=t)
df["address"] = street
df["url"] = url
df

,price,bedrooms,bathrooms,surface,address,url
0,"$2,750",3 bd,2 ba,999 sq ft,5 Donnybrook Rd,https://www.apartments.com/5-donnybrook-rd-bos...


## Automate

https://www.apartments.com/5-donnybrook-rd-boston-ma/rhyk6z3/

In [101]:
# initialize dataframe
t0 = ["address","price","bedrooms","bathrooms","surface","url"]
df = pd.DataFrame(columns=t0)
# df = pd.read_csv("boston.csv",index_col=None)
df

,address,price,bedrooms,bathrooms,surface,url


In [102]:
import clipboard
from helper import geosearch
url = clipboard.paste()
# print(url)
driver.get(url)
info_elems = driver.find_elements(by=By.CLASS_NAME, value="rentInfoDetail")
infos = [i.text for i in info_elems]
address_elem = driver.find_element(by=By.CLASS_NAME, value="propertyAddressContainer")
address = address_elem.text.split("\n")[0]
infos.insert(0,address)
infos.insert(5,url)
df1 = pd.DataFrame([infos],columns=t0)
df2 = geosearch(df1)
df = pd.concat([df,df2],ignore_index=True)
df.to_csv("boston.csv",index=None)
df

,address,price,bedrooms,bathrooms,surface,url,location,point,latitude,longitude,altitude
0,"5 Donnybrook Rd, Boston, MA 02135","$2,750",3 bd,2 ba,999 sq ft,https://www.apartments.com/5-donnybrook-rd-bos...,"(5, Donnybrook Road, Faneuil, Brighton, Boston...","(42.35408248177119, -71.15858979568584, 0.0)",42.354082,-71.15859,0.0
